In [1]:
import pandas as pd
import os
import json
from key_word_extraction import *

In [2]:
dir_path = os.path.dirname(os.path.realpath("__file__"))
json_result = {}

In [3]:
import os
file_names = []
for file in os.listdir(dir_path):
    if file.endswith(".json") and "company_breakdown" not in file and "job_skill_distribution" not in file and "company_skill_distribution" not in file and "city_job_distribution" not in file:
        file_names.append(os.path.join(dir_path, file))
        
file_names

['/home/ec2-user/web_scrape_data/data_6_24_boston.json',
 '/home/ec2-user/web_scrape_data/data_6_24_chicago.json',
 '/home/ec2-user/web_scrape_data/data_6_24_dc.json',
 '/home/ec2-user/web_scrape_data/data_6_24_ny.json',
 '/home/ec2-user/web_scrape_data/data_6_24_seattle.json',
 '/home/ec2-user/web_scrape_data/data_6_24_sf.json']

In [4]:
def get_job_details_city(df):
    jobs = df.shape[0]
    

In [5]:
total_series = []
total_df = []

job_details_city = {}

for file in file_names:

    city = file.split("_")[-1].split(".")[0]
    df = pd.read_json(file, orient="records")
    df = df.dropna(subset=["details"])
    raw_result = extract_key_word_count(df)
    
    #Get number of jobs listed per city
    job_details_city[city] = df.shape[0]
    
    #used for getting company and skill distribution
    raw_result["city"] = city
    raw_result = raw_result.drop(columns=["details"])
    total_df.append(raw_result)
    
    #Get key words distribution
    result = raw_result.skill.value_counts()
    total_series.append(result)
    json_result[city] = result.to_dict()
    
total_result = total_series[0]
total_series.pop()
for series in total_series:
    total_result = total_result.add(series, fill_value=0)
json_result["total"] = total_result.to_dict()   

In [6]:
df.head()

,companyName,date,details,location,rating,summary,title,url
0,Pinterest,8 days ago,About Pinterest:\nMillions of people across th...,"San Francisco, CA 94103 (South of Market area)",4.2,5+ years of leadership experience managing a t...,"Head of Analytics and Data Science, Monetization",https://www.indeed.com/viewjob?jk=cee3b86d39e0...
1,NaN,30+ days ago,"August 31, 2018\nData scientist\nJob Requireme...","San Francisco, CA",NaN,"(Ability to write functions, clean and efficie...",Data scientist,https://www.indeed.com/viewjob?jk=cae371ba7263...
2,Seneca Family of Agencies,9 days ago,Overview:\n\nThe Data Analyst supports the dat...,"Oakland, CA 94601 (Harrington area)",4.2,Applying basic statistical concepts and data v...,Data Analyst,https://www.indeed.com/viewjob?jk=81a6c658e9e3...
3,University of California Berkeley,30+ days ago,Data Scientist (0499U) 17919 About Berkeley\n\...,"Berkeley, CA 94720",4.2,"Demonstrated experience with data mining, mach...",Data Scientist (0499U) 17919,https://www.indeed.com/viewjob?jk=89058aed18d4...
4,Allscripts,10 days ago,Welcome to Allscripts! Our Mission is to be th...,"San Francisco, CA",3.6,2+ years of experience in a data analytics rol...,Real-World Evidence Data Analyst - Life Scienc...,https://www.indeed.com/viewjob?jk=863bc3bc75e9...


#### Key Word Distribution Save

In [7]:
with open(dir_path + "/" + "job_skill_distribution.json", "w") as file:
    file.write(json.dumps(json_result, indent=4, sort_keys=True))
    
with open("/var/www/html/data/" + "job_skill_distribution.json", "w") as file:
    file.write(json.dumps(json_result, indent=4, sort_keys=True))

#### Number of jobs listed per city Save

In [8]:
with open(dir_path + "/" + "city_job_distribution.json", "w") as file:
    file.write(json.dumps(job_details_city, indent=4, sort_keys=True))
    
with open("/var/www/html/data/" + "city_job_distribution.json", "w") as file:
    file.write(json.dumps(job_details_city, indent=4, sort_keys=True))

#### Company Skill DF

In [9]:
total_df = pd.concat(total_df)
comapny_skill_dict = extract_company(total_df)


with open(dir_path + "/" + "company_skill_distribution.json", "w") as file:
    file.write(json.dumps(comapny_skill_dict, indent=4, sort_keys=True))
    
with open("/var/www/html/data/" + "company_skill_distribution.json", "w") as file:
    file.write(json.dumps(comapny_skill_dict, indent=4, sort_keys=True))

### company Breakdown

In [10]:
total_df = total_df.drop(columns=["skill"]).drop_duplicates()
company_breakdown = total_df.companyName.value_counts().to_dict()

In [11]:
with open(dir_path + "/" + "company_breakdown.json", "w") as file:
    file.write(json.dumps(company_breakdown, indent=4, sort_keys=True))
    
with open("/var/www/html/data/" + "company_breakdown.json", "w") as file:
    file.write(json.dumps(company_breakdown, indent=4, sort_keys=True))